In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import torchvision.datasets as datasets
import torchvision.transforms as transforms


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#define model parameters
args = {
    "NUM_EPOCHS": 90, #from original paper
    "BATCH_SIZE": 128,
    "MOMENTUM": 0.9,
    "LR_DECAY": 0.0005,
    "LR_INIT": 0.01,
    "IMAGE_DIM": 227,
    "NUM_CLASSES": 1000
}

In [4]:
#import imagenet dataset
transform = transforms.Compose([
    transforms.Resize((args["IMAGE_DIM"], args["IMAGE_DIM"])),
    transforms.ToTensor()]
)
train_dataset = datasets.ImageNet(root='data', split='train', download=True, transform=transform)
test_dataset = datasets.ImageNet(root='data', split='val', download=True, transform=transform)

#load data
train_loader = data.DataLoader(train_dataset, batch_size=args["BATCH_SIZE"], shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=args["BATCH_SIZE"], shuffle=False)



RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in data.

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return x
    
    

In [ ]:
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [ ]:
def test(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)


In [ ]:
model = AlexNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=args["LR_INIT"], momentum=args["MOMENTUM"], weight_decay=args["LR_DECAY"])
criterion = nn.CrossEntropyLoss()

# Data loading code
